In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error, r2_score

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
    
data = fetch_california_housing(as_frame=True)

In [22]:
# from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder(sparse=False)
# _categ = data[['nationality','sport']]
# _transformed = pd.DataFrame(ohe.fit_transform(_categ))
# _transformed.index = _categ.index
# X = pd.concat([data[['height', 'weight', 'gold', 'silver', 'bronze']], _transformed], axis=1)

In [23]:
print(data.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [24]:
data.data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MedInc      20640 non-null  float64
 1   HouseAge    20640 non-null  float64
 2   AveRooms    20640 non-null  float64
 3   AveBedrms   20640 non-null  float64
 4   Population  20640 non-null  float64
 5   AveOccup    20640 non-null  float64
 6   Latitude    20640 non-null  float64
 7   Longitude   20640 non-null  float64
dtypes: float64(8)
memory usage: 1.3 MB


In [25]:
data.target

0        4.526
1        3.585
2        3.521
3        3.413
4        3.422
         ...  
20635    0.781
20636    0.771
20637    0.923
20638    0.847
20639    0.894
Name: MedHouseVal, Length: 20640, dtype: float64

In [34]:
model = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
printmd('''
|metrics|value|
|--|--|
|**RMSE** | %.05f |
|**R2** | %.05f |
''' % (mean_squared_error(predictions,y_test),
r2_score(predictions, y_test)))


|metrics|value|
|--|--|
|**RMSE** | 0.51407 |
|**R2** | 0.37607 |
